In [17]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 

In [ ]:
%%nuclio cmd
pip install kfserving==0.2.0 --upgrade
pip install azure
pip install numpy
pip install xgboost
pip install --upgrade git+https://github.com/fchollet/keras

Requirement already up-to-date: kfserving==0.2.0 in /User/.pythonlibs/lib/python3.6/site-packages (0.2.0)
  Using cached https://files.pythonhosted.org/packages/6e/fc/2cab119f679648b348b8940de0dd744a1f0ee99c690aa2ef6072f050816c/kubernetes-10.0.1-py2.py3-none-any.whl
    100% |████████████████████████████████| 276kB 10.2MB/s ta 0:00:01
kfp 0.1.36 has requirement kubernetes<=9.0.0,>=8.0.0, but you'll have kubernetes 10.0.1 which is incompatible.
azure 4.0.0 has requirement azure-storage-blob~=1.3, but you'll have azure-storage-blob 12.1.0 which is incompatible.
  Found existing installation: kubernetes 9.0.0
    Uninstalling kubernetes-9.0.0:
      Successfully uninstalled kubernetes-9.0.0
  Found existing installation: azure-storage-blob 1.5.0
    Uninstalling azure-storage-blob-1.5.0:
      Successfully uninstalled azure-storage-blob-1.5.0
  Using cached https://files.pythonhosted.org/packages/25/f4/a307ed89014e9abb5c5cfc8ca7f8f797d12f619f17a6059a6fd4b153b5d0/azure_storage_blob-1.5.0-p

In [9]:
import kfserving
import os
import numpy as np
from typing import List, Any

import xgboost as xgb

In [10]:
BOOSTER_FILE = "model.bst"

class RNNModel(kfserving.KFModel):
    def __init__(self, name: str, model_dir: str, booster: xgb.XGBModel = None):
        super().__init__(name)
        self.name = name
        self.model_dir = model_dir
        if not booster is None:
            self._booster = booster
            self.ready = True

    def load(self):
        model_file = os.path.join(
            kfserving.Storage.download(self.model_dir), BOOSTER_FILE)
        self._booster = xgb.Booster(model_file=model_file)
        self.ready = True

    def predict(self, body: List) -> List:
        try:
            # Use of list as input is deprecated see https://github.com/dmlc/xgboost/pull/3970
            dmatrix = xgb.DMatrix(body)
            result: xgb.DMatrix = self._booster.predict(dmatrix)
            return result.tolist()
        except Exception as e:
            raise Exception("Failed to predict %s" % e)


In [25]:
# nuclio: end-code
from mlrun import new_model_server
import requests

In [34]:
fn = new_model_server('Credit-srv', models={'Credit_v1': '/User/Credit_run1/mlrun/data/1'}, model_class='RNNModel')
fn.with_v3io('User','~/') 

In [35]:
addr = fn.deploy(project='Credit-demo')

[mlrun] 2019-12-05 10:56:02,294 deploy started
[nuclio.deploy] 2019-12-05 10:56:03,460 (info) Building processor image


INFO:nuclio.deploy:(info) Building processor image


[nuclio.deploy] 2019-12-05 10:56:09,566 (info) Build complete


INFO:nuclio.deploy:(info) Build complete


[nuclio.deploy] 2019-12-05 10:56:17,717 (info) Function deploy complete


INFO:nuclio.deploy:(info) Function deploy complete


[nuclio.deploy] 2019-12-05 10:56:17,729 done updating higgs-srv, function address: 172.29.142.121:32519


INFO:nuclio.deploy:done updating higgs-srv, function address: 172.29.142.121:32519


In [36]:
# Seldon protocol event
event_seldon = {"data": {"ndarray": [[5], [10]]}}
csel = str(event_seldon).replace("\'", "\"")

In [37]:
resp = requests.put(addr + '/predict/Credit_v1', data=csel)
print(resp.content)

b'{"data": {"ndarray": [[0.34849661588668823, 0.1382627934217453, 0.06415506452322006, 0.06415506452322006, 0.06415506452322006, 0.06415506452322006, 0.06415506452322006, 0.06415506452322006, 0.06415506452322006, 0.06415506452322006], [0.34849661588668823, 0.1382627934217453, 0.06415506452322006, 0.06415506452322006, 0.06415506452322006, 0.06415506452322006, 0.06415506452322006, 0.06415506452322006, 0.06415506452322006, 0.06415506452322006]]}}'
